# HRPFP: previsão de fluxo com histórico + tempo real
Implementação em PyTorch do modelo híbrido descrito em Ouyang et al. (2020) — Appl. Sci. 10(11):3788 — que combina codificadores LSTM para histórico e dados em tempo real (p.2, p.5).

### Visão geral do modelo (p.2, p.5, p.7–8)
- Histórico: usa 7 passos temporais anteriores (mesmo horário em 7 dias) com 22 atributos: dia da semana, seção de tempo, index/lat/lon da parada e 17 contagens de POI (p.2, p.8).
- Tempo real: usa as 4 janelas anteriores do dia em curso para prever os 2 próximos intervalos (p.2).
- Fusão: projeções lineares para igualar formas (Equações 12–13), SoftMax para normalização e soma dos vetores de histórico e tempo real (Equações 14–15, p.5–7).
- Decodificador: três camadas densas (Equações 16–18, p.7–8) e perda MSE entre previsto e real.
- Antes do uso prático, o artigo seleciona o melhor raio de serviço para POIs com XGBoost e RMSE (p.7, p.10).

### Imports e configuração (Equações 6–11 para LSTM, p.5)
Usamos PyTorch para as LSTMs/MLP, SciKit-Learn para utilidades e NumPy para geração sintética.

In [1]:
import math
import random
from dataclasses import dataclass
from typing import List, Tuple

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error

# Garantir reprodutibilidade simples
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cpu')

### Hiperparâmetros alinhados ao artigo (p.2, p.7–8)
Sequências de 7 (histórico) e 4 (tempo real), vetor de 22 atributos, projeção para 50 dimensões e decodificador de três camadas.

In [2]:
@dataclass
class HRPFPConfig:
    history_seq_len: int = 7
    realtime_seq_len: int = 4
    pred_horizon: int = 2
    history_feature_dim: int = 22
    history_hidden: int = 50  # saída 1x50 do codificador de histórico (p.8)
    realtime_hidden: int = 32
    combined_dim: int = 50
    decoder_hidden1: int = 32
    decoder_hidden2: int = 16
    max_flow: float = 300.0  # usado para normalizar contagens

config = HRPFPConfig()
config


HRPFPConfig(history_seq_len=7, realtime_seq_len=4, pred_horizon=2, history_feature_dim=22, history_hidden=50, realtime_hidden=32, combined_dim=50, decoder_hidden1=32, decoder_hidden2=16, max_flow=300.0)

### Geração de dados sintéticos (p.2, p.7–8)
Sem o dataset de Beijing, criamos um conjunto artificial com a mesma estrutura: 7 dias de histórico com 22 atributos normalizados (dia-da-semana + seção temporal + estação/lat/lon + 17 POIs), 4 medições em tempo real (contagens) e alvo em 2 passos futuros. As contagens são escaladas por um `max_flow` para representar a normalização Min–Max indicada pelo artigo (p.8).

In [4]:
def _build_feature_vector(dow: float, time_section: float, station_idx: float, lon: float, lat: float, poi_counts: np.ndarray) -> np.ndarray:
    # 5 atributos espaço-temporais + 17 POIs = 22 dim (p.8)
    base = [dow, time_section, station_idx, lon, lat]
    return np.concatenate([np.array(base, dtype=np.float32), poi_counts.astype(np.float32)], axis=0)

def _simulate_flow(base_level: float, time_section: int, noise_scale: float = 8.0) -> float:
    # Padrão diário com pico matutino/vespertino e ruído
    morning_peak = 1.3 * math.exp(-((time_section - 7) ** 2) / 18)
    evening_peak = 1.1 * math.exp(-((time_section - 17) ** 2) / 20)
    seasonal = 0.2 * math.sin(time_section / 24 * 2 * math.pi)
    return max(5.0, base_level * (1 + morning_peak + evening_peak + seasonal) + np.random.normal(0, noise_scale))

def generate_synthetic_sample(cfg: HRPFPConfig, rng: np.random.Generator) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    station_idx = rng.integers(0, 20)
    time_section = int(rng.integers(0, 24))
    base_day = int(rng.integers(0, 30))
    lon = 0.5 + rng.normal(0, 0.05)
    lat = 0.5 + rng.normal(0, 0.05)
    poi_counts = rng.uniform(0, 1, 17)
    base_level = 40 + 30 * poi_counts.mean() + 5 * (station_idx / 20)

    history_seq = []
    for i in range(cfg.history_seq_len):
        dow = ((base_day - (cfg.history_seq_len - i)) % 7) / 6
        feat = _build_feature_vector(dow, time_section / 23, station_idx / 20, lon, lat, poi_counts)
        history_seq.append(feat)
    history_seq = np.stack(history_seq)

    realtime_seq = []
    targets = []
    for step in range(cfg.realtime_seq_len + cfg.pred_horizon):
        flow = _simulate_flow(base_level, time_section + step) / cfg.max_flow
        if step < cfg.realtime_seq_len:
            realtime_seq.append([flow])
        else:
            targets.append(flow)
    realtime_seq = np.array(realtime_seq, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)
    return history_seq.astype(np.float32), realtime_seq, targets

def generate_dataset(cfg: HRPFPConfig, n_samples: int, seed: int = 0):
    rng = np.random.default_rng(seed)
    history, realtime, targets = [], [], []
    for _ in range(n_samples):
        h, r, t = generate_synthetic_sample(cfg, rng)
        history.append(h)
        realtime.append(r)
        targets.append(t)
    return np.stack(history), np.stack(realtime), np.stack(targets)

raw_history, raw_realtime, raw_targets = generate_dataset(config, n_samples=600, seed=SEED)
raw_history.shape, raw_realtime.shape, raw_targets.shape


((600, 7, 22), (600, 4, 1), (600, 2))

### Dataset e DataLoaders com normalização simples (p.8)
O artigo enfatiza a normalização das 22 features (Min–Max). As contagens em tempo real e os alvos já estão divididos por `max_flow`; as features simuladas estão em [0,1] por construção. Se desejar, basta substituir o gerador pela leitura do dataset real de AFC/POI seguindo o mesmo formato tensorial.

In [5]:
class HRPFPSyntheticDataset(Dataset):
    def __init__(self, history: np.ndarray, realtime: np.ndarray, targets: np.ndarray):
        self.history = torch.from_numpy(history)
        self.realtime = torch.from_numpy(realtime)
        self.targets = torch.from_numpy(targets)

    def __len__(self):
        return len(self.history)

    def __getitem__(self, idx):
        return self.history[idx], self.realtime[idx], self.targets[idx]

full_dataset = HRPFPSyntheticDataset(raw_history, raw_realtime, raw_targets)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
len(train_dataset), len(val_dataset)


(480, 120)

### Ajuste de raio de serviço para POIs (p.7, p.10, Eq.19)
O artigo escolhe o raio que minimiza o RMSE ao combinar POIs via XGBoost. Abaixo está uma versão simplificada/toy usando as 17 features de POI sintéticas: escalonamos as contagens por um fator dependente do raio e testamos diferentes valores com XGBoostRegressor.

In [6]:
from xgboost import XGBRegressor

def estimate_best_radius(poi_matrix: np.ndarray, target: np.ndarray, radii=(100, 200, 300, 400, 500)):
    scores = []
    for r in radii:
        weight = math.exp(-abs(r - 300) / 200)  # simula decaimento de relevância fora do ótimo (~300 m, ver p.10)
        X = poi_matrix * weight
        model = XGBRegressor(
            max_depth=4, learning_rate=0.05, n_estimators=120, subsample=0.8, colsample_bytree=0.9,
            objective='reg:squarederror', random_state=SEED
        )
        model.fit(X, target)
        preds = model.predict(X)
        rmse = math.sqrt(mean_squared_error(target, preds))
        scores.append((r, rmse))
    scores.sort(key=lambda x: x[1])
    return scores[0][0], scores

poi_features = raw_history[:, 0, 5:]  # 17 POIs do primeiro passo histórico apenas para demo
best_radius, radius_scores = estimate_best_radius(poi_features, raw_targets.mean(axis=1))
best_radius, radius_scores[:3]


(100,
 [(100, 0.04946607125723421),
  (200, 0.04946607125723421),
  (300, 0.04946607125723421)])

### Arquitetura HRPFP em PyTorch (Equações 12–18, p.5–8)
Segue a sequência do artigo: codificador LSTM de histórico (1×50), codificador LSTM de tempo real, projeções lineares + SoftMax para combinar vetores e decodificador de 3 camadas com Sigmoid na segunda camada. A perda utilizada é MSE entre `P` e o alvo (p.7).

In [7]:
class HRPFPModel(nn.Module):
    def __init__(self, cfg: HRPFPConfig):
        super().__init__()
        self.history_encoder = nn.LSTM(input_size=cfg.history_feature_dim, hidden_size=cfg.history_hidden, batch_first=True)
        self.realtime_encoder = nn.LSTM(input_size=1, hidden_size=cfg.realtime_hidden, batch_first=True)
        self.history_proj = nn.Linear(cfg.history_hidden, cfg.combined_dim)
        self.realtime_proj = nn.Linear(cfg.realtime_hidden, cfg.combined_dim)
        self.decoder1 = nn.Linear(cfg.combined_dim, cfg.decoder_hidden1)
        self.decoder2 = nn.Linear(cfg.decoder_hidden1, cfg.decoder_hidden2)
        self.output = nn.Linear(cfg.decoder_hidden2, cfg.pred_horizon)

    def forward(self, history_x: torch.Tensor, realtime_x: torch.Tensor):
        # history_x: (batch, seq_len, 22)
        _, (h_n, _) = self.history_encoder(history_x)
        hist_vec = self.history_proj(h_n[-1])
        _, (r_n, _) = self.realtime_encoder(realtime_x)
        real_vec = self.realtime_proj(r_n[-1])
        hist_soft = torch.softmax(hist_vec, dim=-1)
        real_soft = torch.softmax(real_vec, dim=-1)
        info = hist_soft + real_soft
        d1 = self.decoder1(info)
        d2 = torch.sigmoid(self.decoder2(d1))
        return self.output(d2)


### Treino rápido em dados sintéticos (p.7)
Otimizamos a MSE (equivalente a minimizar RMSE² do Eq.19) com Adam. Com dados reais, ajustaríamos épocas e early stopping; aqui rodamos poucas iterações para demonstrar o fluxo.

In [8]:
model = HRPFPModel(config).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

train_history = []
val_history = []

for epoch in range(15):
    model.train()
    running_loss = 0.0
    for hist_batch, rt_batch, tgt_batch in train_loader:
        hist_batch = hist_batch.to(device)
        rt_batch = rt_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        optimizer.zero_grad()
        preds = model(hist_batch, rt_batch)
        loss = criterion(preds, tgt_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * len(hist_batch)

    train_rmse = math.sqrt(running_loss / len(train_loader.dataset))

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for hist_batch, rt_batch, tgt_batch in val_loader:
            preds = model(hist_batch.to(device), rt_batch.to(device))
            loss = criterion(preds.cpu(), tgt_batch)
            val_loss += loss.item() * len(hist_batch)
    val_rmse = math.sqrt(val_loss / len(val_loader.dataset))

    train_history.append(train_rmse)
    val_history.append(val_rmse)
    if epoch % 5 == 0 or epoch == 14:
        print(f'Epoch {epoch:02d} — train RMSE: {train_rmse:.4f}, val RMSE: {val_rmse:.4f}')

train_history[-1], val_history[-1]


Epoch 00 — train RMSE: 0.5979, val RMSE: 0.5060
Epoch 05 — train RMSE: 0.0947, val RMSE: 0.0955
Epoch 10 — train RMSE: 0.0933, val RMSE: 0.0929
Epoch 14 — train RMSE: 0.0933, val RMSE: 0.0927


(0.09330896592144161, 0.09271418168162561)

### Avaliação qualitativa das previsões (p.12–13)
Mostramos alguns pares real×previsto (contagens desscaladas). No artigo, as Figuras 9–12 comparam curvas de tempo; aqui apenas listamos valores para verificar que o modelo captura tendências.

In [9]:
model.eval()
with torch.no_grad():
    sample_hist, sample_rt, sample_tgt = next(iter(val_loader))
    preds = model(sample_hist.to(device), sample_rt.to(device)).cpu()

for i in range(5):
    real_counts = (sample_tgt[i] * config.max_flow).numpy()
    pred_counts = (preds[i] * config.max_flow).numpy()
    print(f'Exemplo {i}: Real {real_counts.round(1)} | Previsto {pred_counts.round(1)}')


Exemplo 0: Real [116.3  96.8] | Previsto [99.7 97.9]
Exemplo 1: Real [108.4 106.2] | Previsto [99.7 97.9]
Exemplo 2: Real [53.5 54.9] | Previsto [99.6 97.9]
Exemplo 3: Real [53.9 57.5] | Previsto [99.6 97.9]
Exemplo 4: Real [129.  107.5] | Previsto [99.8 97.9]


### Próximos passos sugeridos
- Substituir o gerador sintético pelo dataset real de AFC/POI, mantendo os tensores `history_x`, `realtime_x` e alvos normalizados.
- Implementar a coleta de POIs por raio real via API (Figura 3, p.8) e alimentar `estimate_best_radius` com as agregações reais.
- Reproduzir as comparações com séries temporais e LSTM puro (Seção 4, p.10–13) para validar contra os números do artigo.